In [1]:
import pandas as pd
import numpy as np
import cchardet
import re
from textrank4zh import TextRank4Keyword, TextRank4Sentence
import jieba
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("chinese_news.csv",encoding="gb18030")
df = df.dropna(subset = ['content',"title"])
df = df.reset_index(drop=True)
# 截断数据，小规模测试
df = df[:201]

In [3]:
class textSummary(object):
    def __init__(self,N):
        super().__init__()
        self.N = N
    
    def drop_brackets_content(self,sentence):
        """
        清除括号中内容
        """
        pattern  = "（[\s\S]*?）"
        res = re.sub(pattern,"",sentence)
        return res
    
    def clean_sentence(self,sentence):
        """
        去掉空格和换行符
        """
        clean_sentence = sentence.replace("\r\n","").replace("　　","")
        return clean_sentence
    
    
    def clean_and_drop_brackets_content(self,sentence):
        """
        清除括号内容和空格
        """
        return self.clean_sentence(self.drop_brackets_content(sentence))
    
    
    def save_key_sentence(self,sentence,title):
        """
        通过textrank获取关键词
        """
        sentence_score_list = []
        tr4s = TextRank4Sentence()
        tr4s.analyze(text=sentence, lower=True, source = 'all_filters')
        
        sentence_score_list.append((title+"。",0.5))

        for item in tr4s.get_key_sentences(num=100):
            sentence_score_list.append((item.sentence,item.weight))
        sentence_score_list.sort(key=lambda x:-x[1])
        print(sentence_score_list)
        return sentence_score_list
    
    
    def output_summary(self,sentence_score_list):
        """
        按照句子的分数取出句子进行拼接，直到逼近字数N为止
        """
        sentence = ""
        for key,value in sentence_score_list:
            if len(sentence) < self.N:
                sentence += key
            else:
                break
        return sentence

In [4]:
def get_input_artical_summary(_index):
    """
    只写一种：根据_index指定chinese_news的文章id进行摘要提取
    """
    df = pd.read_csv("chinese_news.csv",encoding="gb18030")
    df = df.dropna(subset = ['content',"title"])
    df = df.reset_index(drop=True)
    
    ts = textSummary(100)
    
    # 截断数据，小规模测试
    df = df[:201]
    
    return ts.output_summary(ts.save_key_sentence(ts.clean_and_drop_brackets_content(df["content"][_index]),ts.clean_and_drop_brackets_content(df["title"][_index])))

In [11]:
get_input_artical_summary(27)

[('火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力。', 0.5), ('路透社称，梅决定提前大选，又未能让保守党在大选中获得绝对多数，已经让英国陷入自一年前“脱欧”公投以来最深刻的政治危机中', 0.048921632854693795), ('英国《每日电讯报》称，议会选举败北后，梅对14日“格伦费尔塔”火灾的无情和迟钝反应令她陷入巨大的政治危险', 0.048780382733447036), ('“梅承认做得不够好”，BBC17日称，为平息怒火，梅当天抽出2小时，在唐宁街会见灾民和志愿者，并主持了一场政府应对火灾的会议', 0.0450318834841547), ('路透社17日称，火灾发生后，英国女王伊丽莎白二世和她的孙子威廉王子16日赴火灾发生地探望灾民和志愿者，女王17日又在自己91岁官方生日庆典上主持了1分钟的默哀仪式，并针对英国近来发生的数起事故“罕见”地呼吁民众“在哀伤中团结起来”', 0.04430888243052025), ('专栏作家、前保守党议员帕里斯认为，现在，梅应对火灾的行动表明，她缺乏判断力，“若无法重建公众信任，这个首相当不久”', 0.04228843473460915), ('这场集会原本是抗议梅率领的保守党与爱尔兰民主统一党谈判联合组阁的，但后来加入了许多对梅应对火灾不力不满的民众', 0.04114364248512619), ('英国“天空新闻网”17日称，梅在事发后视察火灾现场但未慰问灾民备受指责', 0.04088802856772735), ('《星期日电讯报》18日也引述一些保守党“脱欧”派资深人士的话说，如果梅在即将展开的“脱欧”谈判中，背离原来的“硬脱欧”计划，他们就会立即对梅的领导权提出挑战', 0.039988488550341664), ('《星期日泰晤士报》18日称，在梅领导的保守党内，人们对梅的信心不断下降，现在一些人甚至已经向她发出最后通牒，要求她在10天内证明她拥有自己所说的“领导能力”，否则就会采取行动赶她下台', 0.03954459213855338), ('报道称，除英国女王外，工党领袖科尔宾也在第一时间去了现场并探望幸存者，他们的做法与梅形成“鲜明对比”', 0.03764200730034719), ('报道透露，至少12名保守党议员已打算致函代表保守党后座议员的组织——1

'火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力。路透社称，梅决定提前大选，又未能让保守党在大选中获得绝对多数，已经让英国陷入自一年前“脱欧”公投以来最深刻的政治危机中英国《每日电讯报》称，议会选举败北后，梅对14日“格伦费尔塔”火灾的无情和迟钝反应令她陷入巨大的政治危险'

### 改动库的源码，操作需谨慎
- 上面的代码中我们可以看到我们的句子是没有分割标点符号的
- 所以查找源代码发现使用的是str.split的方式，这样确实会导致分隔符的标点符号的丢失
- 所以对源码进行改造，改成re.split的方式

```python
import re
class SentenceSegmentation(object):
    """ 分句 """
    
    def __init__(self, delimiters=util.sentence_delimiters):
        """
        Keyword arguments:
        delimiters -- 可迭代对象，用来拆分句子
        """
        self.delimiters = set([util.as_text(item) for item in delimiters])
    
    def segment(self, text):
        res = [util.as_text(text)]
        
        util.debug(res)
        util.debug(self.delimiters)
        
        # 原始的str.split的方式
        # for sep in self.delimiters:
        #     text, res = res, []
        #     for seq in text:
        #         res += seq.split(sep)
        # res = [s.strip() for s in res if len(s.strip()) > 0]
        
        # new 使用re.split的方式
        sentences = re.split(r"([。!！?？；;\s+])", text)[:-1]
        sentences.append("")
        res = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
        return res
```

In [5]:
# 对源码改动后的输出展示，可以看到出现了分割标点
get_input_artical_summary(27)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\waws\AppData\Local\Temp\jieba.cache
Loading model cost 0.500 seconds.
Prefix dict has been built successfully.


[('火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力。', 0.5), ('路透社称，梅决定提前大选，又未能让保守党在大选中获得绝对多数，已经让英国陷入自一年前“脱欧”公投以来最深刻的政治危机中。', 0.04764709847300687), ('英国《每日电讯报》称，议会选举败北后，梅对14日“格伦费尔塔”火灾的无情和迟钝反应令她陷入巨大的政治危险。', 0.04743574509428193), ('“梅承认做得不够好”，BBC17日称，为平息怒火，梅当天抽出2小时，在唐宁街会见灾民和志愿者，并主持了一场政府应对火灾的会议。', 0.04373474311076242), ('路透社17日称，火灾发生后，英国女王伊丽莎白二世和她的孙子威廉王子16日赴火灾发生地探望灾民和志愿者，女王17日又在自己91岁官方生日庆典上主持了1分钟的默哀仪式，并针对英国近来发生的数起事故“罕见”地呼吁民众“在哀伤中团结起来”。', 0.0431413544755193), ('专栏作家、前保守党议员帕里斯认为，现在，梅应对火灾的行动表明，她缺乏判断力，“若无法重建公众信任，这个首相当不久”。', 0.04118733757527311), ('这场集会原本是抗议梅率领的保守党与爱尔兰民主统一党谈判联合组阁的，但后来加入了许多对梅应对火灾不力不满的民众。', 0.03997833528398352), ('英国“天空新闻网”17日称，梅在事发后视察火灾现场但未慰问灾民备受指责。', 0.03980038085156115), ('《星期日电讯报》18日也引述一些保守党“脱欧”派资深人士的话说，如果梅在即将展开的“脱欧”谈判中，背离原来的“硬脱欧”计划，他们就会立即对梅的领导权提出挑战。', 0.038896868519705904), ('《星期日泰晤士报》18日称，在梅领导的保守党内，人们对梅的信心不断下降，现在一些人甚至已经向她发出最后通牒，要求她在10天内证明她拥有自己所说的“领导能力”，否则就会采取行动赶她下台。', 0.03846851191833982), ('报道称，除英国女王外，工党领袖科尔宾也在第一时间去了现场并探望幸存者，他们的做法与梅形成“鲜明对比”。', 0.03833177128250652), ('报道透露，至少12名保守党议员已打算致函代表保守党后座

'火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力。路透社称，梅决定提前大选，又未能让保守党在大选中获得绝对多数，已经让英国陷入自一年前“脱欧”公投以来最深刻的政治危机中。英国《每日电讯报》称，议会选举败北后，梅对14日“格伦费尔塔”火灾的无情和迟钝反应令她陷入巨大的政治危险。'